<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Fetching the Nodes Locations on the Metr-LA Dataset</h2>
</center>

---

In this notebook the street and kilometrage of each node in the dataset are fetched.

Note: the extraction of the geolocation of the nodes is not deterministic, refer to [link](../data/metr-la/structured/node_locations.pkl) for the nodes streets and kilometrages extracted for the experiment.

In [2]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [3]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [4]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

# 1 Loading the Data
In this section the data is loaded

In [5]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'metr-la')

In [6]:
from src.data.data_extraction import get_adjacency_matrix


# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

In [7]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_metr_la.csv'),
    has_header=True)

In [8]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

# 2 Fetch the Nodes Locations and Kilometrage
In this section the node locations and kilometrages is fetched.

# 2.1 Fetch the Nodes Roads
The roads of the nodes are fetched by geopy according to their latitude and longitude.

In [9]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geocoder = Nominatim(user_agent='metr-la')

geocode = RateLimiter(
    geocoder.geocode,
    min_delay_seconds=.05,
    return_value_on_exception=None)

In [10]:
import pandas as pd

def get_road(row: pd.Series) -> str:
    data = geocoder.reverse((row['latitude'], row['longitude'])).raw
    road = data['address']['road']
    return road

In [15]:
locations_df['road'] = locations_df.apply(get_road, axis=1)

In [16]:
set(road for road in locations_df['road'].values)

{'Arroyo Seco Parkway',
 'East 4th Street',
 'Fletcher Drive',
 'Foothill Freeway',
 'Franklin Avenue',
 'Glendale Freeway',
 'Golden State Freeway',
 'Hollywood Freeway',
 'Metro G Line Bikeway',
 'North Brand Boulevard',
 'North Western Avenue',
 'North Wilton Place',
 'San Diego Freeway',
 'Santa Ana Freeway',
 'Sherman Way',
 'Silver Lake Boulevard',
 'US 101',
 'Ventura Freeway'}

Fixing wrong assignment of coordinates

In [17]:
for row in locations_df.itertuples():
    if row.road in ['East 4th Street', 'Sherman Way', 'US 101']:
        print(f'{row.road}:',
              'coordinates', 
              f'({row.latitude}, {row.longitude})')

US 101: coordinates (34.06491, -118.25126)
East 4th Street: coordinates (34.04301, -118.21724)
Sherman Way: coordinates (34.20112, -118.47361)
US 101: coordinates (34.1527, -118.3754)
US 101: coordinates (34.15133, -118.37456)


By searching these coordinates, it can be observed that they actually refer to specific highways, hence we manually change this wrongly assigned reference

In [18]:
import pandas as pd


def change_road_name(row: pd.Series) -> str:
    if row['road'] == 'East 4th Street':
        return 'Golden State Freeway'

    elif row['road'] == 'Sherman Way':
        return 'San Diego Freeway'

    elif row['road'] == 'US 101':
        return 'Hollywood Freeway'

    return row['road']

In [19]:
locations_df['road'] = locations_df.apply(change_road_name, axis=1)

In [20]:
set(road for road in locations_df['road'].values)

{'Arroyo Seco Parkway',
 'Fletcher Drive',
 'Foothill Freeway',
 'Franklin Avenue',
 'Glendale Freeway',
 'Golden State Freeway',
 'Hollywood Freeway',
 'Metro G Line Bikeway',
 'North Brand Boulevard',
 'North Western Avenue',
 'North Wilton Place',
 'San Diego Freeway',
 'Santa Ana Freeway',
 'Silver Lake Boulevard',
 'Ventura Freeway'}

## 2.2 Get the Road Beginnings

In order to get the kilometrages of the nodes, the road beginnings are computed from [ArcGIS official website](https://www.arcgis.com/home/webmap/viewer.html?featurecollection=https%3A%2F%2Fgeo.dot.gov%2Fserver%2Frest%2Fservices%2FHosted%2FCalifornia_2018_PR%2FFeatureServer%3Ff%3Djson%26option%3Dfootprints&supportsProjection=true&supportsJSONP=true)

In [21]:
road_beginnings = {
    'Arroyo Seco Parkway': (34.06261, -118.24863),
    'Foothill Freeway': (34.317596, -118.481173),
    'Glendale Freeway': (34.207526, -118.215157),
    'Golden State Freeway': (34.060422, -118.213057),
    'Hollywood Freeway': (34.151159, -118.373795),
    'San Diego Freeway': (34.294368, -118.469921),
    'Santa Ana Freeway': (34.06312,	-118.247073),
    'Ventura Freeway': (34.147306, -118.160817)}

In [22]:
road_beginnings.update({
    'Silver Lake Boulevard': (34.0954, -118.2745),
    'Fletcher Drive': (34.1121, -118.2526),
    'Vine Street': (34.1016, -118.3269),
    'North Western Avenue': (34.1042, -118.3085),
    'North Brand Boulevard': (34.1512, -118.2544)
})

road_beginnings.update({
    'Avenue 43': (34.0909, -118.2116),  # Near intersection with Griffin Ave
    'Fern Lane': (34.1122, -118.2140)   # Small residential road near Mt Washington
})

road_beginnings.update({
    'Franklin Avenue': (34.1032, -118.3276),         # Near Hollywood Blvd intersection
    'Metro G Line Bikeway': (34.1886, -118.4494),    # Near Balboa Blvd in the Valley
    'North Wilton Place': (34.0989, -118.3105)       # Near Melrose Ave in Hollywood
})

## 2.3 Get Nodes Kilometrages and Save Results
Finally, the kilometrage of the nodes is computed by their distance from the coordinates of the beginnings of the roads.

For each node a dictionary is computed containing a tuple with the name of the road it is part of and its kilometrage on the road.

In [23]:
import pandas as pd
from geopy.distance import distance


def get_kilometrage(
    row: pd.Series,
    ) -> float:
    road = row['road']
    road_beginning_coordinates = road_beginnings[road]
    coordinates = (row['latitude'], row['longitude'])
    return distance(road_beginning_coordinates, coordinates).km

In [24]:
locations_df['kilometrage'] = locations_df.apply(get_kilometrage, axis=1)

In [25]:
locations_df.head()

,sensor_id,latitude,longitude,road,kilometrage
index,,,,,
0,773869,34.15497,-118.31829,Ventura Freeway,14.547154
1,767541,34.11621,-118.23799,Glendale Freeway,10.345752
2,767542,34.11641,-118.23819,Glendale Freeway,10.327808
3,717447,34.07248,-118.26772,Hollywood Freeway,13.112986
4,717446,34.07142,-118.26572,Hollywood Freeway,13.329056


In [26]:
road_locations = {}

for row in locations_df.itertuples():
    node_id = row.sensor_id
    road = row.road
    kilometrage = row.kilometrage
    road_locations[node_id] = (road, kilometrage)

In [27]:
import os
import pickle

os.makedirs(os.path.join(BASE_DATA_DIR, 'structured'), exist_ok=True)

with open(
      os.path.join(BASE_DATA_DIR, 'structured', 'node_locations.pkl'), 
      'wb') as f:
    pickle.dump(road_locations, f)

## 2.4 Route Recommendation

In [ ]:
# --- STGNN Prediction, Clustering, MCTS Routing, and Agent Explanation ---

import numpy as np
from sklearn.cluster import DBSCAN
from collections import defaultdict

# 1. Predict future traffic using STGNN (placeholder)
def predict_future_traffic(graph_data, model, steps=12):
    """
    Predict traffic for the next `steps` time intervals.
    Replace this with your actual STGNN inference code.
    """
    # Example: return np.random.rand(num_nodes, steps)
    return model.predict(graph_data, steps=steps)

# 2. Cluster nodes using DBSCAN
def cluster_nodes(locations_df, eps=0.01, min_samples=3):
    coords = locations_df[['latitude', 'longitude']].values
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    locations_df['zone'] = clustering.labels_
    return locations_df

# 3. MCTS for route search (simplified placeholder)
class MCTSNode:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.children = []
        self.visits = 0
        self.value = 0

def mcts_search(start, end, adj_matrix, traffic_pred, max_depth=10, simulations=100):
    """
    Simulate different routes from start to end using MCTS.
    Returns the best route found.
    """
    # Placeholder: Use BFS for demonstration, replace with real MCTS logic
    from collections import deque
    queue = deque([[start]])
    best_route = None
    best_score = float('inf')
    while queue:
        path = queue.popleft()
        if len(path) > max_depth:
            continue
        current = path[-1]
        if current == end:
            # Score: sum of predicted traffic on this path
            score = sum(traffic_pred[node] for node in path)
            if score < best_score:
                best_score = score
                best_route = path
            continue
        for neighbor, connected in enumerate(adj_matrix[current]):
            if connected and neighbor not in path:
                queue.append(path + [neighbor])
    return best_route, best_score

# 4. Agent logic
class RoutingAgent:
    def __init__(self, adj_matrix, traffic_pred, locations_df):
        self.adj_matrix = adj_matrix
        self.traffic_pred = traffic_pred
        self.locations_df = locations_df

    def plan_routes(self, start, end):
        route, score = mcts_search(start, end, self.adj_matrix, self.traffic_pred)
        return route, score

    def explain_route(self, route, score):
        explanation = {
            "route": route,
            "total_predicted_delay": score,
            "robustness": "N/A (placeholder)",
            "congestion_points": [node for node in route if self.traffic_pred[node] > np.percentile(self.traffic_pred, 80)]
        }
        return explanation

# --- Example Usage ---

# Assume you have a trained STGNN model and graph_data
# traffic_pred = predict_future_traffic(graph_data, stgnn_model, steps=1).flatten()
# For demonstration, use random predictions:
traffic_pred = np.random.rand(len(locations_df))

# Cluster nodes
locations_df = cluster_nodes(locations_df)

# Initialize agent
agent = RoutingAgent(adj_matrix, traffic_pred, locations_df)

# Plan and explain a route (example: node 0 to node 10)
route, score = agent.plan_routes(0, 10)
explanation = agent.explain_route(route, score)

print("Best Route:", route)
print("Explanation:", explanation)

Best Route: [0, 199, 117, 10]
Explanation: {'route': [0, 199, 117, 10], 'total_predicted_delay': 1.3615702178868245, 'robustness': 'N/A (placeholder)', 'congestion_points': []}


In [29]:
from keplergl import KeplerGl
import pandas as pd

# Extract coordinates for the route
route_coords = locations_df.loc[route, ['sensor_id', 'latitude', 'longitude']].copy()
route_coords['order'] = range(len(route_coords))

# Create KeplerGl map
map_ = KeplerGl(height=500)
map_.add_data(data=route_coords, name="Route")

# Display the map
map_

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Route': {'index': [0, 199, 117, 10], 'columns': ['sensor_id', 'latitude', 'longitude', 'order'…